[Reference: R for Data Science Chapter 13](http://r4ds.had.co.nz/relational-data.html)

# 13.1 Introduction

* multiple tables of data are called relational data because it is the relations, not just the individual datasets, that are important
* Relations are always defined between a pair of tables.

* **Mutating joins**, which add new variables to one data frame from matching observations in another.
* **Filtering joins**, which filter observations from one data frame based on whether or not they match an observation in the other table.
* **Set operations**, which treat observations as if they were set elements.

## 13.1.1 Prerequisites

In [2]:
library(tidyverse)
library(nycflights13)

# 13.2 nycflights13

Dataset containts four tables:
* `airlines` lets you look up the full carrier name from its abbreviated code
* `airports` gives information about each airport, identified by the `faa` airport code
* `planes` gives information about each plane, identified by its `tailnum`
* `weather` gives the weather at each NYC airport for each hour

<img src="http://r4ds.had.co.nz/diagrams/relational-nycflights.png", width=520, height=360>

* `flights` connects to `planes` via a single variable, `tailnum`.
* `flights` connects to `airlines` through the `carrier` variable.
* `flights` connects to `airports` in two ways: via the `origin` and `dest` variables.
* `flights` connects to `weather` via `origin` (the location), and `year`, `month`, `day` and `hour` (the time).

# 13.3 Keys

A key is a variable (or set of variables) that uniquely identifies an observation.

There are two types of keys:
* A primary key uniquely identifies an observation in its own table. For example, `planes$tailnum` is a primary key because it uniquely identifies each plane in the planes table.
* A foreign key uniquely identifies an observation in another table. For example, the `flights$tailnum` is a foreign key because it appears in the flights table where it matches each flight to a unique plane.

In [3]:
# check whether we identified the primary keys
planes %>% 
  count(tailnum) %>% 
  filter(n > 1) # if the number is 0, it is fine!

tailnum,n


If a table lacks a primary key, it’s sometimes useful to add one with `mutate()` and `row_number()`. That makes it easier to match observations if you’ve done some filtering and want to check back in with the original data. This is called a **surrogate key**.

Relationship:
* 1-to-1: special case of 1-to-many
* 1-to-many / many-to-1: each flight has one plane, but each plane has many flights
* many-to-many: airports and airlines / each airline flies to many airpots and each airports hosts many airlines

# 13.4 Mutating joins

In [4]:
# 扱いやすいように変数を減らす
flights2 <- flights %>% 
  select(year:day, hour, origin, dest, tailnum, carrier)
head(flights2)

year,month,day,hour,origin,dest,tailnum,carrier
2013,1,1,5,EWR,IAH,N14228,UA
2013,1,1,5,LGA,IAH,N24211,UA
2013,1,1,5,JFK,MIA,N619AA,AA
2013,1,1,5,JFK,BQN,N804JB,B6
2013,1,1,6,LGA,ATL,N668DN,DL
2013,1,1,5,EWR,ORD,N39463,UA


In [5]:
# 航空会社の名前を上のtableに追加したい
head(airlines, 3)

carrier,name
9E,Endeavor Air Inc.
AA,American Airlines Inc.
AS,Alaska Airlines Inc.


In [6]:
flights2 %>%
  select(-origin, -dest) %>% 
  left_join(airlines, by = "carrier") %>% head(3)

year,month,day,hour,tailnum,carrier,name
2013,1,1,5,N14228,UA,United Air Lines Inc.
2013,1,1,5,N24211,UA,United Air Lines Inc.
2013,1,1,5,N619AA,AA,American Airlines Inc.


In [7]:
# Another way (not recommended)
flights2 %>%
  select(-origin, -dest) %>% 
  mutate(name = airlines$name[match(carrier, airlines$carrier)]) %>% head(3)

year,month,day,hour,tailnum,carrier,name
2013,1,1,5,N14228,UA,United Air Lines Inc.
2013,1,1,5,N24211,UA,United Air Lines Inc.
2013,1,1,5,N619AA,AA,American Airlines Inc.


## 13.4.1 Understanding joins

In [8]:
x <- tribble(
  ~key, ~val_x,
     1, "x1",
     2, "x2",
     3, "x3"
)
y <- tribble(
  ~key, ~val_y,
     1, "y1",
     2, "y2",
     4, "y3"
)

## 13.4.2 Inner join

[Image in Textbook](http://r4ds.had.co.nz/diagrams/join-inner.png)

## 13.4.3 Outer joins

[Image in Textbook](http://r4ds.had.co.nz/diagrams/join-outer.png)

## 13.4.4 Duplicate keys

[Image in Textbook: One table has dublicate keys](http://r4ds.had.co.nz/diagrams/join-one-to-many.png)

[Image in Textbook: Both tables have duplicate keys](http://r4ds.had.co.nz/diagrams/join-many-to-many.png)

## 13.4.5 Defining the key columns

`by = c("a" = "b")` will match variable `a` in table `x` to variable `b` in table `y`

In [9]:
flights2 %>% 
  left_join(airports, c("dest" = "faa")) %>% head(3)

year,month,day,hour,origin,dest,tailnum,carrier,name,lat,lon,alt,tz,dst,tzone
2013,1,1,5,EWR,IAH,N14228,UA,George Bush Intercontinental,29.98443,-95.34144,97,-6,A,America/Chicago
2013,1,1,5,LGA,IAH,N24211,UA,George Bush Intercontinental,29.98443,-95.34144,97,-6,A,America/Chicago
2013,1,1,5,JFK,MIA,N619AA,AA,Miami Intl,25.79325,-80.29056,8,-5,A,America/New_York


## 13.4.7 Other implementations

baseやSQLとの対応

# 13.5 Filtering joins

* [`semi_join(x, y)`](http://r4ds.had.co.nz/diagrams/join-semi.png) keeps all observations in `x` that have a match in `y`.
* [`anti_join(x, y)`](http://r4ds.had.co.nz/diagrams/join-anti.png) drops all observations in `x` that have a match in `y`.

In [10]:
# Semi-joins are useful for matching filtered summary tables back to the original rows
top_dest <- flights %>%
  count(dest, sort = TRUE) %>%
  head(5)
top_dest

dest,n
ORD,17283
ATL,17215
LAX,16174
BOS,15508
MCO,14082


In [11]:
flights %>% 
  filter(dest %in% top_dest$dest) %>% select(dest)　%>% table()

.
  ATL   BOS   LAX   MCO   ORD 
17215 15508 16174 14082 17283 

In [12]:
# 上と同じことをする
flights %>% 
  semi_join(top_dest, by="dest") %>% select(dest)　%>% table()

.
  ATL   BOS   LAX   MCO   ORD 
17215 15508 16174 14082 17283 

In [13]:
# Anti-joins are useful for diagnosing join mismatches. 
flights %>%
  anti_join(planes, by = "tailnum") %>%
  count(tailnum, sort = TRUE) %>% head(3)

tailnum,n
NA,2512
N725MQ,575
N722MQ,513


# 13.6 Join problems

# 13.7 Set operations

In [14]:
df1 <- tribble(
  ~x, ~y,
   1,  1,
   2,  1
)
df2 <- tribble(
  ~x, ~y,
   1,  1,
   1,  2
)

In [15]:
intersect(df1, df2) # return only observations in both x and y

x,y
1,1


In [16]:
union(df1, df2) # return unique observations in x and y

x,y
1,2
2,1
1,1


In [17]:
setdiff(df1, df2) # return observations in x, but not in y

x,y
2,1
